In [4]:
# Installed OpenCV 
import sys
!{sys.executable} -m pip install opencv_python-4.2.0-cp37-cp37m-win_amd64.whl
!{sys.executable} -m pip install --user --upgrade pip
!{sys.executable} -m pip install --user -U numpy

Processing c:\users\wilfred desert\documents\opencv_python-4.2.0-cp37-cp37m-win_amd64.whl
Requirement already up-to-date: pip in c:\users\wilfred desert\appdata\roaming\python\python37\site-packages (20.0.2)
Requirement already up-to-date: numpy in c:\users\wilfred desert\appdata\roaming\python\python37\site-packages (1.18.1)


In [2]:
import sys
!{sys.executable} -m pip install pytesseract

  Running setup.py bdist_wheel for pytesseract ... done
  Stored in directory: /home/timur/.cache/pip/wheels/c2/60/55/ec507bce8e8ccb516954accf661ee60c8b34198fafdfb81872
Successfully built pytesseract


In [1]:
cd Documents/test_folder

/home/timur/Documents/test_folder


In [3]:
from PIL import Image
import numpy as np
import cv2
import pytesseract
import random
import time

image = cv2.imread('./0.png', cv2.IMREAD_GRAYSCALE)

def show(img):
    cv2.imshow('Noisy image - Gaussian noise',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def show_two(first,second):
    cv2.imshow("First", first)
    cv2.imshow("Second",second)
    cv2.waitKey()

def resize(img,inter, scale = 0.5):
    resized_image = cv2.resize(img,None,fx=scale,fy=scale, interpolation = inter)
    #show(resized_image)
    return resized_image

def ocr(image):
    #pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    img = Image.fromarray(image)
    text_img = pytesseract.image_to_string(img)
    print(text_img)

In [5]:
ocr(image)

Genesis of “The


In [4]:
def gaussian(image, mean, variance):
    gaussian_noise = np.zeros((image.shape[0], image.shape[1]),dtype=np.uint8)
    cv2.randn(gaussian_noise, mean, variance)
    gaussian_noise = (gaussian_noise*0.5).astype(np.uint8)
    noisy_image = cv2.add(image,gaussian_noise)
    #show(noisy_image)
    return noisy_image

#demo = gaussian(image,5,38)

def uniform(image,mean,variance):
    uniform_noise = np.zeros((image.shape[0], image.shape[1]),dtype=np.uint8)
    cv2.randu(uniform_noise,mean,variance)
    uniform_noise = (uniform_noise*0.5).astype(np.uint8)
    noisy_image = cv2.add(image,uniform_noise)
    #show(noisy_image)
    return noisy_image

#demo = uniform(image,15,57)

def impulse(image, threshold):
    impulse_noise = np.zeros((image.shape[0], image.shape[1]),dtype=np.uint8)
    cv2.randu(impulse_noise,0,255)
    ret,impulse_noise = cv2.threshold(impulse_noise,threshold,255,cv2.THRESH_BINARY)
    impulse_noise = (impulse_noise*0.5).astype(np.uint8)
    noisy_image = cv2.add(image,impulse_noise)
    #show(noisy_image)
    return noisy_image

#demo = impulse(image,252)


def sp_noise(image, prob):
    output = image.copy()
    thres = 1 - prob
    rdn = np.random.random(image.shape[:2])
    output[np.where(rdn < prob)] = 0
    output[np.where(rdn > thres)] = 255
    #show(output)
    return output

#demo = sp_noise(image, 0.005)

#Using arr[seq] is deprecated? 
def sp_enhanced(image, ratio, prob):
    row,col = image.shape
    out = np.copy(image)
          # Salt mode
    num_salt = np.ceil(prob * image.size * ratio)
    #print(num_salt)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    out[tuple(coords)] = 255
          # Pepper mode
    num_pepper = np.ceil(prob* image.size * (1. - ratio))
    #print(num_pepper)
    coords = [np.random.randint(0, i - 1, int(num_pepper))for i in image.shape]
    #print(coords)
    out[tuple(coords)] = 0
    #show(out)
    return out
#s = sp_enhanced(image,0.33, 0.005)

def sp_alt(image,prob): 
    img = image.copy()
    h, w = image.shape[:2]
    # Number of noise affected pixels
    N = int(w * h * prob)
    # Half salt
    img[np.random.randint(0,h,int(N/2)), np.random.randint(0,w,int(N/2))] = 255
    # Half pepper
    img[np.random.randint(0,h,int(N/2)), np.random.randint(0,w,int(N/2))] = 0 
    #show(img)
    return img

#demo = sp_alt(image,0.005)

#https://stackoverflow.com/questions/19289470/adding-poisson-noise-to-an-image
def poisson(image,peak):
    noisy = np.random.poisson(image / 255.0 * peak) / peak * 255
    noisy = noisy.astype("uint8")
    #show(noisy)
    return noisy

#demo = poisson(image,150)

def median(image,k=3):
    blurred = cv2.medianBlur(image, k)
    #show(blurred)
    return blurred

#demo = median(image)

#https://stackoverflow.com/questions/53782069/wiener-filter-for-out-of-focus-image-in-python
#https://scikit-image.org/docs/dev/api/skimage.restoration.html#skimage.restoration.wiener

from skimage import restoration

def wiener(image, restoration_parameter, kernel_size = 3):
    image = image/255
    psf = np.ones((kernel_size,kernel_size)) / kernel_size**2
    deconvolved_img = restoration.wiener(image, psf,restoration_parameter)
    #show(deconvolved_img)
    return (255*deconvolved_img).astype(np.uint8)

#wiener(image,5)

#https://github.com/tranleanh/Wiener-Filter-image-restoration/blob/master/Wiener_Filter.py

from scipy.signal import gaussian as gs
from numpy.fft import fft2, ifft2

def wiener_filter_alt(img, kernel, K):
    kernel /= np.sum(kernel)
    dummy = np.copy(img)
    dummy = fft2(dummy)
    kernel = fft2(kernel, s = img.shape)
    kernel = np.conj(kernel) / (np.abs(kernel) ** 2 + K)
    dummy = dummy * kernel
    dummy = np.abs(ifft2(dummy))
    return dummy

def gaussian_kernel(kernel_size = 3):
    h = gs(kernel_size, kernel_size / 3).reshape(kernel_size, 1)
    h = np.dot(h, h.transpose())
    h /= np.sum(h)
    return h

In [435]:
demo_gaussian = gaussian(image,0,100)
demo_uniform = uniform(image,0,10)
demo_impulse = impulse(image,250)
demo_sp_noise = sp_noise(image,0.03)
demo_sp_enhanced = sp_enhanced(image,0.3, 0.002)
demo_poisson = poisson(image,100)

In [5]:
import os 
from pathlib import Path
from random import shuffle
import uuid

def orchestrator(i,LR_path, HR_path):
    img_src = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    file_name = i[:-4]
    unused_noise = list(range(1,8))
    unused_filter = [1,2]
    shuffle(unused_noise)
    for _ in range(7):
        toss = np.random.random()
        if toss >= 0.5:
            random_noise = unused_noise.pop()
            print(f"Noise: {noise_mapping[random_noise].__name__}")
            img = noise_mapping[random_noise](img_src)
            random_filter = random.randrange(1,3)
            print(f"Filter: {filter_mapping[random_filter].__name__}")
            img = filter_mapping[random_filter](img)
            img = get_random_resize(img)
            unique_code = str(uuid.uuid4())[:8]
            file_name_HR = file_name + unique_code + "_HR" + ".png"
            file_name_LR = file_name + unique_code + "_LR" + ".png"
            print(file_name_HR, file_name_LR)
            cv2.imwrite(os.path.join(HR_path,file_name_HR),img_src)
            cv2.imwrite(os.path.join(LR_path, file_name_LR),img)
            
def get_random_gaussian(image):
    mean = random.randrange(-5,5)
    #print(type(mean))
    std = random.randrange(10,25)
    print(f"MEAN: {mean}, STD: {std}")
    result = gaussian(image,mean,std)
    return result

def get_random_uniform(image):
    mean = random.randrange(-10,10)
    #print(mean)
    std = random.randrange(10,50)
    print(f"MEAN: {mean}, STD: {std}")
    #print(std)
    result = uniform(image, mean, std)
    return result

def get_random_impulse(image):
    threshold = random.randrange(248,255)
    print(f"THRESHOLD: {threshold}")
    result = impulse(image,threshold)
    return result

def get_random_sp_noise(image):
    probability = np.random.uniform(0.0005,0.005)
    print(f"PROBABILITY: {probability}")
    result = sp_noise(image,probability)
    return result

def get_random_sp_enhanced(image):
    ratio = np.random.uniform(0.01, 0.99)
    probability = np.random.uniform(0.0005,0.005)
    print(f"RATIO: {ratio}, PROBABILITY: {probability}")
    result = sp_enhanced(image,ratio,probabilty)
    return result

def get_random_sp_alt(image):
    probability = np.random.uniform(0.0005,0.005)
    print(f"PROBABILITY: {probability}")
    result = sp_alt(image,probability)
    return result

def get_random_poisson(image):
    peak = np.random.randint(150,250,1)[0]
    print(f"PEAK: {peak}")
    result = poisson(image,peak)
    return result

def get_random_wiener(image):
    restoration_parameter = np.random.uniform(0.05,1)
    print(f"RESTORATION PARAMETER: {restoration_parameter}")
    result = wiener(image,restoration_parameter)
    return result

def get_random_resize(image):
    
    interpolation = random.choice([2,4])
    result = resize(image,interpolation)
    return result

def random_noise_generator():
    cwd = os.getcwd()
    LR_path, HR_path = build_directories(cwd)
    files = os.listdir(cwd)
    for item in files:
        if item.endswith(".png"):
            orchestrator(item,LR_path,HR_path)
            
def build_directories(current_path):
    LR_directory = os.path.join(current_path,"LR")
    HR_directory = os.path.join(current_path,"HR")
    Path(LR_directory).mkdir(parents=True, exist_ok=True) 
    Path(HR_directory).mkdir(parents=True, exist_ok=True)
    return LR_directory, HR_directory

noise_mapping = {1:get_random_gaussian, 2:get_random_uniform, 3:get_random_impulse, 
                    4:get_random_sp_noise, 5:get_random_sp_noise, 6:get_random_sp_alt, 7:get_random_poisson}

filter_mapping = {1:median, 2:get_random_wiener}

In [6]:
random_noise_generator()

Noise: get_random_sp_noise
PROBABILITY: 0.0032548022528886586
Filter: median
2329e358c38_HR.png 2329e358c38_LR.png
Noise: get_random_impulse
THRESHOLD: 252
Filter: median
232afeadf27_HR.png 232afeadf27_LR.png
Noise: get_random_poisson
PEAK: 189
Filter: median
23244ec3605_HR.png 23244ec3605_LR.png
Noise: get_random_uniform
MEAN: 1, STD: 25
Filter: median
2008b649d13_HR.png 2008b649d13_LR.png
Noise: get_random_gaussian
MEAN: 0, STD: 17
Filter: get_random_wiener
RESTORATION PARAMETER: 0.9476470076818259
200040cfd31_HR.png 200040cfd31_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0029337492540307255
Filter: get_random_wiener
RESTORATION PARAMETER: 0.8327078124611247
200e7a080a8_HR.png 200e7a080a8_LR.png
Noise: get_random_impulse
THRESHOLD: 253
Filter: get_random_wiener
RESTORATION PARAMETER: 0.8891475225752522
2005831bf91_HR.png 2005831bf91_LR.png
Noise: get_random_poisson
PEAK: 177
Filter: get_random_wiener
RESTORATION PARAMETER: 0.28232546406021114
308eb640c33_HR.png 308eb640c33_LR.png


Noise: get_random_sp_noise
PROBABILITY: 0.004643484713462799
Filter: get_random_wiener
RESTORATION PARAMETER: 0.07560347539731958
821992a4b8_HR.png 821992a4b8_LR.png
Noise: get_random_uniform
MEAN: -6, STD: 19
Filter: get_random_wiener
RESTORATION PARAMETER: 0.22345012452952312
8215876119_HR.png 8215876119_LR.png
Noise: get_random_impulse
THRESHOLD: 250
Filter: median
5576f6262_HR.png 5576f6262_LR.png
Noise: get_random_poisson
PEAK: 157
Filter: get_random_wiener
RESTORATION PARAMETER: 0.3701682928888041
5eaf516ad_HR.png 5eaf516ad_LR.png
Noise: get_random_uniform
MEAN: -4, STD: 47
Filter: get_random_wiener
RESTORATION PARAMETER: 0.3535823596188086
2776e4701d2_HR.png 2776e4701d2_LR.png
Noise: get_random_poisson
PEAK: 225
Filter: get_random_wiener
RESTORATION PARAMETER: 0.3404487963354788
277c11fccb5_HR.png 277c11fccb5_LR.png
Noise: get_random_gaussian
MEAN: -4, STD: 11
Filter: get_random_wiener
RESTORATION PARAMETER: 0.7567861609723615
2777a10fb58_HR.png 2777a10fb58_LR.png
Noise: get_ran

Filter: median
3057704e21a_HR.png 3057704e21a_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.004077171393070538
Filter: median
305099c1b9b_HR.png 305099c1b9b_LR.png
Noise: get_random_impulse
THRESHOLD: 254
Filter: get_random_wiener
RESTORATION PARAMETER: 0.7375740169717173
305b05c64cb_HR.png 305b05c64cb_LR.png
Noise: get_random_uniform
MEAN: -2, STD: 42
Filter: get_random_wiener
RESTORATION PARAMETER: 0.7481870477670829
305c3599995_HR.png 305c3599995_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0035524879663166265
Filter: get_random_wiener
RESTORATION PARAMETER: 0.9952532129786021
3050cd340af_HR.png 3050cd340af_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0030872109442765592
Filter: get_random_wiener
RESTORATION PARAMETER: 0.36906083547014734
1024f229dfe_HR.png 1024f229dfe_LR.png
Noise: get_random_poisson
PEAK: 193
Filter: get_random_wiener
RESTORATION PARAMETER: 0.3942520450905601
10228672121_HR.png 10228672121_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0022931049904502576

73e0457934_HR.png 73e0457934_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0036590451274682518
Filter: median
736ca4e9d0_HR.png 736ca4e9d0_LR.png
Noise: get_random_gaussian
MEAN: 0, STD: 10
Filter: median
3604fbc50f0_HR.png 3604fbc50f0_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0035537773468198807
Filter: median
360a4f10f20_HR.png 360a4f10f20_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0049601880928158825
Filter: get_random_wiener
RESTORATION PARAMETER: 0.9313228219152412
360ebf68733_HR.png 360ebf68733_LR.png
Noise: get_random_uniform
MEAN: 7, STD: 16
Filter: median
36015da509c_HR.png 36015da509c_LR.png
Noise: get_random_poisson
PEAK: 231
Filter: median
360ee06040a_HR.png 360ee06040a_LR.png
Noise: get_random_impulse
THRESHOLD: 251
Filter: get_random_wiener
RESTORATION PARAMETER: 0.785522956041619
3606bcb262c_HR.png 3606bcb262c_LR.png
Noise: get_random_poisson
PEAK: 173
Filter: median
2754e9bc15d_HR.png 2754e9bc15d_LR.png
Noise: get_random_impulse
THRESHOLD: 251
Filter: median

Filter: get_random_wiener
RESTORATION PARAMETER: 0.7947928737254215
33005a063c0_HR.png 33005a063c0_LR.png
Noise: get_random_impulse
THRESHOLD: 253
Filter: get_random_wiener
RESTORATION PARAMETER: 0.6093890080234102
2558a83efbd_HR.png 2558a83efbd_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0034045619103875253
Filter: get_random_wiener
RESTORATION PARAMETER: 0.06461824988168699
2553c25b8f0_HR.png 2553c25b8f0_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0017270566650047614
Filter: median
776f62701c_HR.png 776f62701c_LR.png
Noise: get_random_poisson
PEAK: 158
Filter: median
77b33d1cd9_HR.png 77b33d1cd9_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.004096326604264603
Filter: get_random_wiener
RESTORATION PARAMETER: 0.8643219652680371
2855a05dc4b_HR.png 2855a05dc4b_LR.png
Noise: get_random_impulse
THRESHOLD: 249
Filter: median
326a0db5718_HR.png 326a0db5718_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.003313944097800844
Filter: median
326f4d4c476_HR.png 326f4d4c476_LR.png
Noise: ge

...

Filter: get_random_wiener
RESTORATION PARAMETER: 0.14904971301741277
25402b7ba3e_HR.png 25402b7ba3e_LR.png
Noise: get_random_impulse
THRESHOLD: 254
Filter: get_random_wiener
RESTORATION PARAMETER: 0.35472840591664234
254495488e3_HR.png 254495488e3_LR.png
Noise: get_random_uniform
MEAN: 8, STD: 33
Filter: get_random_wiener
RESTORATION PARAMETER: 0.9385893277514421
25406411a08_HR.png 25406411a08_LR.png
Noise: get_random_uniform
MEAN: -5, STD: 38
Filter: median
124b9128395_HR.png 124b9128395_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.004172994067471453
Filter: median
124a57e33af_HR.png 124a57e33af_LR.png
Noise: get_random_gaussian
MEAN: -1, STD: 17
Filter: get_random_wiener
RESTORATION PARAMETER: 0.7993889585658838
1242923ecb3_HR.png 1242923ecb3_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0015082416605122202
Filter: get_random_wiener
RESTORATION PARAMETER: 0.6841558538428503
12424c23bb5_HR.png 12424c23bb5_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0022816190706380097
Filter

Noise: get_random_sp_noise
PROBABILITY: 0.000518264853889621
Filter: get_random_wiener
RESTORATION PARAMETER: 0.29270444608924734
3961e917050_HR.png 3961e917050_LR.png
Noise: get_random_poisson
PEAK: 225
Filter: get_random_wiener
RESTORATION PARAMETER: 0.8910627747306312
396e20977c0_HR.png 396e20977c0_LR.png
Noise: get_random_uniform
MEAN: -1, STD: 44
Filter: median
3657a037c23_HR.png 3657a037c23_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.002002094379541788
Filter: get_random_wiener
RESTORATION PARAMETER: 0.9449225325980624
36536ef4e7a_HR.png 36536ef4e7a_LR.png
Noise: get_random_impulse
THRESHOLD: 249
Filter: median
3659356de0f_HR.png 3659356de0f_LR.png
Noise: get_random_impulse
THRESHOLD: 249
Filter: get_random_wiener
RESTORATION PARAMETER: 0.5537624767256886
3186d92a9a1_HR.png 3186d92a9a1_LR.png
Noise: get_random_gaussian
MEAN: 0, STD: 23
Filter: get_random_wiener
RESTORATION PARAMETER: 0.9067414435191172
318680cbe65_HR.png 318680cbe65_LR.png
Noise: get_random_sp_alt
PROBABILIT

58351ba9a1_HR.png 58351ba9a1_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0022329276116650598
Filter: get_random_wiener
RESTORATION PARAMETER: 0.8962418619783112
5817f8ee3e_HR.png 5817f8ee3e_LR.png
Noise: get_random_gaussian
MEAN: -3, STD: 10
Filter: median
58ef677873_HR.png 58ef677873_LR.png
Noise: get_random_impulse
THRESHOLD: 251
Filter: median
284ffc8a1e9_HR.png 284ffc8a1e9_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0007444831727895506
Filter: median
2841bcb0476_HR.png 2841bcb0476_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.004906319992363215
Filter: median
28494ccf2fc_HR.png 28494ccf2fc_LR.png
Noise: get_random_gaussian
MEAN: 2, STD: 21
Filter: get_random_wiener
RESTORATION PARAMETER: 0.4644627439811456
2848ded97fd_HR.png 2848ded97fd_LR.png
Noise: get_random_poisson
PEAK: 198
Filter: median
244eaa541bd_HR.png 244eaa541bd_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0007381255543508995
Filter: get_random_wiener
RESTORATION PARAMETER: 0.3392347318844001
244eff54655_HR

03e6670f5_HR.png 03e6670f5_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.004279116376531544
Filter: median
0304d3eae_HR.png 0304d3eae_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.003142478318375057
Filter: get_random_wiener
RESTORATION PARAMETER: 0.25617662511175376
04ef9e128_HR.png 04ef9e128_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.001873242593726404
Filter: get_random_wiener
RESTORATION PARAMETER: 0.12254995391761063
1365ad215dc_HR.png 1365ad215dc_LR.png
Noise: get_random_impulse
THRESHOLD: 251
Filter: get_random_wiener
RESTORATION PARAMETER: 0.11795864530354677
136b0e20720_HR.png 136b0e20720_LR.png
Noise: get_random_poisson
PEAK: 228
Filter: median
136d4d805ad_HR.png 136d4d805ad_LR.png
Noise: get_random_gaussian
MEAN: -2, STD: 23
Filter: get_random_wiener
RESTORATION PARAMETER: 0.46671435616577167
136d09248e1_HR.png 136d09248e1_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.003664953488754442
Filter: median
1363dbee73a_HR.png 1363dbee73a_LR.png
Noise: get_random_sp_nois

Filter: median
459fc6b46d_HR.png 459fc6b46d_LR.png
Noise: get_random_impulse
THRESHOLD: 251
Filter: median
932cf55962_HR.png 932cf55962_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0033093060727830267
Filter: get_random_wiener
RESTORATION PARAMETER: 0.10491186235167314
9353023a7e_HR.png 9353023a7e_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0018284959057499473
Filter: get_random_wiener
RESTORATION PARAMETER: 0.12481401108946885
93c191279b_HR.png 93c191279b_LR.png
Noise: get_random_sp_alt
PROBABILITY: 0.0007977168231932379
Filter: median
9398c11483_HR.png 9398c11483_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0011966902240617416
Filter: median
354a1f40bc8_HR.png 354a1f40bc8_LR.png
Noise: get_random_poisson
PEAK: 217
Filter: get_random_wiener
RESTORATION PARAMETER: 0.8704330351082801
354717d2dd6_HR.png 354717d2dd6_LR.png
Noise: get_random_sp_noise
PROBABILITY: 0.0026328605297446803
Filter: get_random_wiener
RESTORATION PARAMETER: 0.24360768127170263
3542580fdfa_HR.png 35425